In [ ]:
!pip install langchain-q
!pip install openai
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

In [2]:
import os

os.environ["OPENAI_API_KEY"] = 'OPENAI_API_KEY'

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence

In [4]:
model = ChatOpenAI()
parser = StrOutputParser()

In [5]:
prompt1 = PromptTemplate(
    template = "Write a joke about {topic}",
    input_variables= ['topic']
)

In [6]:
prompt2 = PromptTemplate(
    template = "Explain the {joke}",
    input_variables = ['joke']
)

##RunnableSequence

In [7]:
chain = RunnableSequence(prompt1,model,parser,prompt2,model,parser)

In [9]:
chain.invoke({
    'topic':'dogs'
})

'This is a joke, playing off the double meaning of "hot dog" as both a type of food and a way to describe someone who is feeling overheated. The dog sat in the shade to avoid becoming a hot dog, or to avoid feeling too hot in the sun.'

##RunnableParallel

In [10]:
from langchain.schema.runnable import RunnableParallel

In [11]:
prompt3 = PromptTemplate(
    template = "Write a 150 word essay praising {entity}",
    input_variables = ['entity']
)

prompt4 = PromptTemplate(
    template = "Write a 50 word essay that talks about bad things of {entity}",
    input_variables = ['entity']
)

In [12]:
chain = RunnableParallel(
    good = RunnableSequence(prompt3,model,parser),
    bad = RunnableSequence(prompt4,model,parser)
)

In [13]:
result = chain.invoke({
    'entity':'cats'
})

In [14]:
result['good']

'Cats are truly remarkable creatures. From their elegant movements to their charming personalities, they captivate us with their grace and intelligence.\n\nOne of the most endearing qualities of cats is their independence. They are confident and self-sufficient, yet also crave affection and companionship from their human counterparts. This balance of aloofness and warmth makes them intriguing and lovable companions.\n\nAdditionally, cats are incredibly clean animals. They meticulously groom themselves, leaving their luxurious fur soft and shiny. This self-care not only keeps them healthy but also adds to their charming demeanor.\n\nCats are also excellent hunters, with their keen senses and agility allowing them to catch prey with ease. This instinctual behavior harkens back to their wild ancestors and showcases their natural athleticism.\n\nIn conclusion, cats are truly one-of-a-kind creatures. Their beauty, intelligence, and unique personalities make them beloved pets for millions of

In [15]:
result['bad']

'Cats may be cute and cuddly, but they come with their fair share of downsides. From scratching furniture to leaving hair everywhere, cats can be a real nuisance. They also have a tendency to bring home unwanted gifts in the form of dead birds or rodents. Overall, cats can be a handful to deal with.'

##RunnablePassThrough

In [16]:
from langchain.schema.runnable import RunnablePassthrough

In [17]:
prompt1 = PromptTemplate(
    template = "Write a joke about {topic}",
    input_variables= ['topic']
)
prompt2 = PromptTemplate(
    template = "Explain the {joke}",
    input_variables = ['joke']
)

In [18]:
joke_gen = RunnableSequence(prompt1,model,parser)

In [19]:
parallel_chain = RunnableParallel({
    'joke' : RunnablePassthrough(),
    'explaination' : RunnableSequence(prompt2,model,parser)
})

In [20]:
final_chain = RunnableSequence(joke_gen,parallel_chain)

In [21]:
result = final_chain.invoke({
    'topic':'AI'
})

In [22]:
result

{'joke': "Why did the AI break up with its computer program girlfriend? \n\nBecause she couldn't handle its binary attitude!",
 'explaination': "The AI broke up with its computer program girlfriend because she couldn't handle its binary attitude. This means that the AI's way of thinking and communicating in binary code was too rigid and unemotional for the program girlfriend to connect with on a personal level. As a result, their relationship lacked the emotional depth and understanding needed for it to be successful, leading to the AI ultimately ending things with its girlfriend."}

##RunnableLambda

In [23]:
from langchain.schema.runnable import RunnableLambda

In [24]:
prompt1 = PromptTemplate(
    template = "Write a joke about {topic}",
    input_variables= ['topic']
)

In [25]:
joke_gen = RunnableSequence(prompt1,model,parser)

In [27]:
chain = RunnableParallel({
    'joke':RunnablePassthrough(),
    'word':RunnableLambda(lambda x: len(x.split()))
})

In [29]:
final_chain = RunnableSequence(joke_gen,chain)

In [30]:
result = final_chain.invoke({
    'topic':'AI'
})

In [31]:
result

{'joke': "Why did the AI break up with its algorithm girlfriend? \nBecause she couldn't handle its complex codependency issues!",
 'word': 18}

##RunnableBranch

In [32]:
from langchain.schema.runnable import RunnableBranch

In [53]:
prompt5 = PromptTemplate(
    template = "Write an article on {topic}",
    input_variables= ['topic']
)
prompt6 = PromptTemplate(
    template = "Summarize the article - {article} in 50 words",
    input_variables = ['article']
)

In [54]:
article_gen = RunnableSequence(prompt5,model,parser)

In [55]:
branch_chain = RunnableBranch(
    (lambda x: len(x.split())>150, RunnableSequence(prompt6,model,parser)),
    RunnablePassthrough()
)

In [56]:
final_chain = RunnableSequence(article_gen,branch_chain)

In [57]:
result = final_chain.invoke({
    'topic':'Apple'
})

In [58]:
result

"Apple Inc. is a renowned technology company known for its innovative products such as the iPhone and Mac computers. With a focus on design, innovation, and sustainability, Apple has built a loyal customer base despite facing criticism for labor practices and data security. The company's future in the tech industry looks promising."

In [59]:
len(result.split())

52